In [ ]:
# 1단계: 환경 초기화, 프로젝트 다운로드 및 경로 설정

# 0. Google Drive 마운트 (결과 저장을 위해)
from google.colab import drive
drive.mount('/content/drive')

# GitHub 릴리스 URL 설정 (소스 코드 및 모델 포함)
ZIP_URL = "https://github.com/kimjongin0325-cyber/ECCV2022-RIFE/releases/download/v3.3/FluidFrames-3.3.zip"

print("--- 1. 프로젝트 파일 다운로드 및 압축 해제 ---")

# 1. Colab 임시 디렉토리(/content)로 이동
%cd /content

# 2. wget을 사용하여 파일 다운로드
print("  > 파일 다운로드 중...")
!wget -q --show-progress {ZIP_URL}

# 3. 압축 해제 (이중 폴더: FluidFrames-3.3/FluidFrames-3.3 생성)
print("  > 압축 해제 중...")
!unzip -q -o FluidFrames-3.3.zip

# 4. 압축 파일 삭제 및 최종 프로젝트 디렉토리로 이동
!rm FluidFrames-3.3.zip
%cd /content/FluidFrames-3.3/FluidFrames-3.3

print("\n✅ 설정 완료: /content/FluidFrames-3.3/FluidFrames-3.3 폴더로 이동되었습니다.")

In [ ]:
!nvidia-smi

Tue Oct 14 11:31:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 2단계: requirements.txt 파일 수정 (Windows 전용 패키지 제거)
import os

file_path = 'requirements.txt'

# 1. 원본 파일 백업 (선택 사항)
if os.path.exists(file_path):
    !cp {file_path} {file_path}.bak
    print(f"✅ 원본 requirements.txt 백업 완료: {file_path}.bak")

# 2. 새로운 내용 정의 (onnxruntime-directml 제거)
new_content = """
# AI
onnx
onnxconverter-common

# GUI
customtkinter

# UTILS
moviepy
opencv-python-headless
Nuitka==2.0.6
"""

# 3. 새로운 내용으로 덮어쓰기
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(new_content.strip() + '\n')

print(f"✅ {file_path} 파일 수정 완료.")

In [ ]:
# 3단계: 필수 패키지 설치 (AI 가속 및 유틸리티)

# 1. 핵심 AI 패키지 설치 (GPU 가속 및 환경 비호환성 해결)
print("1. 핵심 AI 패키지 설치 중...")
!pip install -q torch torchvision onnxruntime numpy tqdm

# 2. 나머지 유틸리티 패키지 설치
print("\n2. 나머지 유틸리티 설치 중...")
!pip install -q -r requirements.txt

print("\n✅ 모든 필수 패키지 설치 완료.")

In [ ]:
# 4단계: 프로그램 최종 실행 및 입/출력 설정

import os
from google.colab import files

# --- 1. 입력 파일 업로드 버튼 ---
print("--- 1. 처리할 입력 영상 파일을 업로드하세요. (파일 선택 버튼이 나타납니다) ---")
uploaded = files.upload()

if uploaded:
    INPUT_FILENAME = list(uploaded.keys())[0]
    INPUT_FILE_PATH = f"/content/{INPUT_FILENAME}"
    print(f"\n✅ 파일 업로드 완료: {INPUT_FILENAME}")
else:
    raise FileNotFoundError("파일 업로드에 실패했습니다. 셀을 다시 실행하거나 파일을 확인해주세요.")

# --- 2. Google Drive 출력 경로 설정 ---
# Drive의 MyDrive 아래 'FluidFrames_Output' 폴더에 저장합니다.
DRIVE_OUTPUT_DIR_BASE = "/content/drive/MyDrive/FluidFrames_Output"
# 입력 파일명 기준 폴더 생성 (예: 'video.mp4' -> 'video_output')
DRIVE_OUTPUT_DIR = f"{DRIVE_OUTPUT_DIR_BASE}/{os.path.splitext(INPUT_FILENAME)[0]}_output" 

# 출력 폴더 생성
!mkdir -p "{DRIVE_OUTPUT_DIR}"

EXECUTION_FILE = "FluidFrames.RIFE.py"


# --- 3. 프로그램 실행 ---
print(f"\n--- 3. 프로그램 실행 시작 ---")
print(f"입력: {INPUT_FILE_PATH}")
print(f"출력: {DRIVE_OUTPUT_DIR}")

# FluidFrames 실행 명령어 (모델 파일은 이미 AI/ 폴더에 있으므로 별도 다운로드 과정 없음)
!python {EXECUTION_FILE} --input_video "{INPUT_FILE_PATH}" --output_dir "{DRIVE_OUTPUT_DIR}"

print("\n✅ 프로그램 실행 완료! 결과 파일은 Google Drive에 저장되었습니다.")

In [ ]:
# 4단계: 프레임 배율 선택, 업스케일링 선택 및 프로그램 최종 실행

import os
from google.colab import files
import re

# =========================================================
# 💡 [핵심 설정 1] 프레임 보간 배율 선택 (필수)
# =========================================================
# 입력 영상의 프레임 수를 몇 배로 만들지 설정합니다.
# 예: 2 -> 30FPS 영상을 60FPS로 만듭니다.
# 예: 4 -> 30FPS 영상을 120FPS로 만듭니다.
INTERPOLATION_FACTOR = 4  # 기본값: 4배속 설정
# =========================================================

# =========================================================
# 💡 [핵심 설정 2] 업스케일링 기능 사용 여부 선택
# =========================================================
ENABLE_UPSCALE = False  # True: 기능 사용 / False: 코드 자동 비활성화
# ... (이전과 동일한 주석 안내) ...
# =========================================================

EXECUTION_FILE = "FluidFrames.RIFE.py"


# --- [추가된 로직] 업스케일링 기능 비활성화 처리 ---
# ... (이전과 동일한 if not ENABLE_UPSCALE: 자동 수정 로직) ...
if not ENABLE_UPSCALE:
    print("--- 업스케일링 기능 비활성화 로직 시작 ---")
    
    file_path = EXECUTION_FILE
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            code_content = f.read()
        
        # 'upscale_icon.png'를 참조하는 코드 라인을 찾아서 주석 처리
        pattern = re.compile(r'^\s*[^#]*upscale_icon\.png.*$', re.MULTILINE)
        new_content = pattern.sub(r'# [DISABLED_UPSCALE] \g<0>', code_content) 

        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(new_content)
        
        print(f"✅ {file_path} 파일 내 'upscale_icon' 참조 라인 주석 처리 완료 (업스케일 버튼 비활성화).")
        
    except Exception as e:
        print(f"⚠️ 경고: 소스 코드 수정 중 오류 발생 ({e}). 수동 확인이 필요할 수 있습니다.")


# --- 1. 입력 파일 업로드 버튼 ---
print("--- 1. 처리할 입력 영상 파일을 업로드하세요. (파일 선택 버튼이 나타납니다) ---")
uploaded = files.upload()

# ... (이전과 동일한 파일 업로드 코드) ...
if uploaded:
    INPUT_FILENAME = list(uploaded.keys())[0]
    INPUT_FILE_PATH = f"/content/{INPUT_FILENAME}"
    print(f"\n✅ 파일 업로드 완료: {INPUT_FILENAME}")
else:
    raise FileNotFoundError("파일 업로드에 실패했습니다. 셀을 다시 실행하거나 파일을 확인해주세요.")

# --- 2. Google Drive 출력 경로 설정 및 생성 ---
DRIVE_OUTPUT_DIR_BASE = "/content/drive/MyDrive/FluidFrames_Output"
DRIVE_OUTPUT_DIR = f"{DRIVE_OUTPUT_DIR_BASE}/{os.path.splitext(INPUT_FILENAME)[0]}_output_{INTERPOLATION_FACTOR}x" # 출력 폴더명에 배율 추가!
!mkdir -p "{DRIVE_OUTPUT_DIR}"


# --- 3. 프로그램 실행 ---
print(f"\n--- 3. 프로그램 실행 시작 ---")
print(f"프레임 보간 배율: {INTERPOLATION_FACTOR}배속")
print(f"입력 파일 경로: {INPUT_FILE_PATH}")
print(f"출력 폴더 경로: {DRIVE_OUTPUT_DIR}")

# FluidFrames 실행 명령어
# [중요]: FluidFrames GUI 내에서 사용자가 직접 '4배속' 버튼 등을 클릭해야 합니다.
# 이 명령어는 GUI를 띄우는 역할을 합니다.
!python {EXECUTION_FILE} --input_video "{INPUT_FILE_PATH}" --output_dir "{DRIVE_OUTPUT_DIR}"

print("\n\n⭐️ 중요 안내: 프로그램 실행 후, GUI 창에서 직접 배율 버튼(2x, 4x 등)을 클릭해야 합니다. ⭐️")
print("✅ 프로그램 실행 완료! 결과 파일은 Google Drive에 저장되었습니다.")